In [1]:
import numpy as np
import geopandas as gpd

from sklearn.cluster import KMeans

In [2]:
blocks_gdf = gpd.read_file("blocks-geodata/nyc_projected_crs.shp")
manhattan_blocks_gdf = blocks_gdf[blocks_gdf["boroname"] == "Manhattan"]

In [3]:
pluto_gdf = gpd.read_file("nyc_mappluto/MapPLUTO.shp")

In [4]:
# Rename BCTCB2020 to lowercase for merging
# Standardize column names and filter for Manhattan
pluto_gdf = pluto_gdf.rename(columns={"BCTCB2020": "bctcb2020"})
manhattan_pluto_gdf = pluto_gdf[pluto_gdf["Borough"] == "MN"].copy()

# Reproject only if not already in EPSG:4326
if manhattan_pluto_gdf.crs != "EPSG:4326":
    manhattan_pluto_gdf = manhattan_pluto_gdf.to_crs("EPSG:4326")

In [5]:
# Aggregate building features per block from pluto_gdf
block_features = (
    manhattan_pluto_gdf.groupby("bctcb2020")
    .agg(
        {
            "BldgArea": "sum",  # total building area
            "NumFloors": "max",  # max number of floors
            "UnitsRes": "sum",  # total residential units
            "UnitsTotal": "sum",  # total units
            "LotArea": "mean",  # average lot area
            "YearBuilt": "mean",  # average year built
            "ComArea": "sum",  # total commercial area
            "NumBldgs": "sum",  # total number of buildings
        }
    )
    .reset_index()
)

# Merge features into gdf
manhattan_blocks_spatial_features_gdf = manhattan_blocks_gdf.merge(
    block_features, on="bctcb2020", how="left"
)

# After merging, rename columns to remove '_y' suffix
manhattan_blocks_spatial_features_gdf = manhattan_blocks_spatial_features_gdf.rename(
    columns={
        "BldgArea_y": "BldgArea",
        "NumFloors_y": "NumFloors",
        "UnitsRes_y": "UnitsRes",
        "UnitsTotal_y": "UnitsTotal",
        "LotArea_y": "LotArea",
        "YearBuilt_y": "YearBuilt",
        "ComArea_y": "ComArea",
        "NumBldgs_y": "NumBldgs",
    }
)

# Fill NaNs with 0 for clustering
feature_cols = [
    "BldgArea",
    "NumFloors",
    "UnitsRes",
    "UnitsTotal",
    "LotArea",
    "YearBuilt",
    "ComArea",
    "NumBldgs",
]
manhattan_blocks_spatial_features_gdf[feature_cols] = (
    manhattan_blocks_spatial_features_gdf[feature_cols].fillna(0)
)

# Create embedding (as a numpy array)
manhattan_blocks_spatial_features_gdf["embedding"] = (
    manhattan_blocks_spatial_features_gdf[feature_cols].apply(
        lambda row: row.values.astype(float), axis=1
    )
)

# Run k-means clustering
n_clusters_spatial = 8
X = np.vstack(manhattan_blocks_spatial_features_gdf["embedding"].values)
kmeans = KMeans(n_clusters=n_clusters_spatial, random_state=42)
manhattan_blocks_spatial_features_gdf["cluster"] = kmeans.fit_predict(X)

In [ ]:
def get_places(lat, lon, api_key, radius, limit=20):
    all_places = []
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lon}",
        "radius": radius,
        "type": "point_of_interest",
        "key": api_key,
    }
    results = []
    next_page_token = None
    while len(results) < limit:
        if next_page_token:
            params["pagetoken"] = next_page_token
        response = requests.get(url, params=params)
        data = response.json()
        results.extend(data.get("results", []))
        next_page_token = data.get("next_page_token")
        if not next_page_token or len(results) >= limit:
            break
    all_places.extend(results[:limit])
    return all_places


def get_place_reviews(place_id, api_key, max_reviews=2):
    """Fetch up to max_reviews review texts for a place using Place Details API."""
    details_url = "https://maps.googleapis.com/maps/api/place/details/json"
    details_params = {"place_id": place_id, "fields": "reviews", "key": api_key}
    details_response = requests.get(details_url, params=details_params)
    details_data = details_response.json()
    reviews_list = details_data.get("result", {}).get("reviews", [])
    review_texts = [r.get("text", "") for r in reviews_list[:max_reviews]]
    return " ".join(review_texts)


def get_radius(row):
    # use sqrt of block area as radius
    if "shape_area" in row:
        lot_area_sqft = row["shape_area"]
        lot_area_sqm = lot_area_sqft * 0.092903
        return math.sqrt(lot_area_sqm) if lot_area_sqm > 0 else 100
    else:
        return 100


api_key = ""
embeddings = []

types = [
    "point_of_interest",
    "restaurant",
    "store",
    "cafe",
    "bar",
    "museum",
    "park",
    "tourist_attraction",
    "transit_station",
    "health",
    "school",
    "library",
    "place_of_worship",
]

print(
    f"Number of blocks: {len(manhattan_blocks_spatial_features_gdf)}"
)  # 3930 blocks in manhattan

# for idx, row in gdf_new.head(1000).iterrows():  # Only process first 1000 blocks-geodata
start_i = 3000
end_i = 3930

for idx, row in manhattan_blocks_spatial_features_gdf.iloc[start_i:end_i].iterrows():
    if idx % 100 == 0:
        print(f"Processing block {idx} ...")
    start_time = time.time()

    # get lat, lon and radius from the block geometry
    centroid = row.geometry.centroid
    centroid_latlon = (
        gpd.GeoSeries([centroid], crs=manhattan_blocks_spatial_features_gdf.crs)
        .to_crs(epsg=4326)
        .geometry[0]
    )
    lat, lon = centroid_latlon.y, centroid_latlon.x
    radius = get_radius(row)

    # Get places for all types
    places = get_places(lat, lon, api_key, radius, limit=20)

    # Count number of each type
    type_counts = {t: 0 for t in types}
    for place in places:
        for t in place.get("types", []):
            if t in type_counts:
                type_counts[t] += 1

    # Top 10 places by user_ratings_total
    top_places = sorted(
        places, key=lambda x: x.get("user_ratings_total", 0), reverse=True
    )[:40]
    top_names = [p.get("name", "") for p in top_places[:10]]

    # For top 10 places with reviews, combine up to 2 reviews into a single string
    top_reviews = []
    for i in range(min(40, len(top_places))):
        p = top_places[i]
        place_id = p.get("place_id")
        if place_id:
            combined_reviews = get_place_reviews(place_id, api_key, max_reviews=5)
            top_reviews.append(combined_reviews)
        if len(top_reviews) >= 10:
            break

    # Build embedding: [type counts] + [top 10 names] + [top 10 combined reviews]
    embedding = [type_counts[t] for t in types] + top_names + top_reviews
    embeddings.append(embedding)
    manhattan_blocks_spatial_features_gdf.at[idx, "places_multimodal_embedding"] = (
        " | ".join([str(x) for x in embedding])
    )

In [ ]:
# Output all data to CSV
manhattan_blocks_spatial_features_gdf.to_csv(
    "../embedding-geodata/manhattan_block_features.csv", index=False
)

# Output all data to GeoJSON
manhattan_blocks_spatial_features_gdf.to_file(
    "../embedding-geodata/manhattan_block_features.geojson", driver="GeoJSON"
)